## 4.2 간단한 분류 모델 구현하기

딥러닝에서 가장 폭넓게 활용되는 분야는 패턴 인식을 통한 영상 처리

ex. 어떤 사진이 고양이인지, 강아지인지, 자동차인지, 비행기인지 판단

-> 패턴을 파악해 여러 종류로 구분하는 작업 : classification(분류)

* 이번 예제는 **털** 과 **날개** 가 있느냐를 기준으로 포유류와 조류를 구분하는 신경망 모델을 만듦

* 핵심 개념에 집중하기 위해 이미지 대신 간단한 이진 데이터 사용

In [1]:
import tensorflow as tf
import numpy as np

C:\Users\soual\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


먼저 tensorflow와 numpy 라이브러리를 import한다. 

numpy : 매우 유명한 수치해석용 파이썬 라이브러리
> 행렬 조작과 연산에 필수적

> 텐서플로 역시 numpy를 긴밀하게 이용

In [13]:
#[털, 날개]
x_data = np.array( [[0,0], [1,0], [1,1], [0,0], [0,0], [0,1]] )

* 학습에 사용할 데이터를 정의

털과 날개가 있느냐를 담은 특징 데이터를 구성한다.

있으면 1, 없으면 0

In [14]:
기타 = [1,0,0]
포유류 = [0,1,0]
조류 = [0,0,1]

각 개체가 실제 어떤 종류인지 나타내는 레이블(분류값) 데이터를 구성 

**-> 앞의 정의한 데이터의 각 개체가 포유류인지 조류인지 그 외인지 기록한 실제 결과값!(y 값이라 생각)**

* 레이블 데이터는 **one-hot encoding(원-핫 인코딩)** 으로 구성
> 데이터가 가질 수 있는 값들을 일렬로 나열한 배열을 만들고, 그 중 표현하려는 값을 뜻하는 인덱스의 원소만 1로 표기하고, 나머지 원소는 모두 0으로 채우는 표기

In [15]:
y_data = np.array([
    [1,0,0], # 기타
    [0,1,0], # 포유류
    [0,0,1], # 기타
    [1,0,0],
    [1,0,0],
    [0,0,1]
])

# [털, 날개] -> [기타, 포유류, 조류]
[0,0] -> [1,0,0] # 기타

[1,0] -> [0,1,0] # 포유류

[1,1] -> [0,0,1] # 조류

[0,0] -> [1,0,0] # 기타

[0,0] -> [1,0,0] # 기타

[0,1] -> [0,0,1] # 조류

와 같은 연관 관계를 갖게 됨

### 신경망 모델 구성 ( 특징 X와 레이블 Y와의 관계 알아내는 모델###

In [16]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

X와 Y에 ground truth(실측값)을 넣어서 학습시킬 것임

X와 Y는 placeholder로 설정

In [17]:
W = tf.Variable(tf.random_uniform([2,3], -1., 1.))
b = tf.Variable(tf.zeros([3]))

신경망을 결정하는 weight와 bias 설정


W : [입력층(특징 수/#feature), 출력층(레이블 수/#layers)] 설정

b : 레이블 수로 설정

-> X에 weight 곱하고 bias 더한 결과를 활성화 함수인 ReLU에 적용하면 신경망 구성 끝!!

In [18]:
L = tf.add(tf.matmul(X, W), b)
L = tf.nn.relu(L)

In [19]:
model = tf.nn.softmax(L)

신경망을 통해 나온 출력값을 softmax 함수를 이용하여 사용하기 쉽게 다듬어 준다.

**softmax : 배열 내의 결과값들을 전체 합이 1이 되도록 만들어 준다.**

-> 전체가 1이므로 각각은 해당 결과의 확률로 해석 가능!
<pre>
[8.04, 2.76, -6.52] -> [0.53, 0.24, 0.23]
<code>

In [20]:
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis = 1))

* loss function 정의

one-hot encoding을 사용하는 대부분의 모델의 경우 **Cross-Entropy 함수(교차 엔트로피)** 사용
> **Cross-Entropy : 예측값과 실제값 사이의 확률 분포 차이를 계산한 값**

* 계산과정 따라가기


1. Y : 실측값 / model : 신경망을 통해 나온 예측값

<pre>
    Y        model
[[1 0 0] [0.1 0.7 0.2]
 [0 1 0] [0.2 0.8 0.0]]
<code>

2. model값에 log를 취한 값을 Y에 곱하면 아래와 같다. 


<pre>
    Y        model            Y * tf.log(model)
[[1 0 0] [0.1 0.7 0.2]    -> [[-1.0 0 0]
 [0 1 0] [0.2 0.8 0.0]]   -> [0 -0.09 0]]
<code>


3. 이제 행별로 값을 다 더한다.

<pre>
    Y*tf.log(model)    reduce_sum(axis = 1)
[[-1.0 0 0]          -> [-1.0, -0.09]
 [0 -0.09 0]]
<code>
 
 
4. 배열 안 값의 평균을 내면 손실값인 cross-entropy 값이 된다.

<pre>
    reduce_sum     reduce_mean
[-1.0, -0.09]   -> -0.545
<code>

### reduce_xxx 함수 ###
* 함수의 차원을 줄여준다.

* 함수 이름의 xxx 부분이 구체적인 축소 방법을 의미한다.

* axis 매개변수로 축소할 차원을 정한다

ex) reduce_sum(<입력 텐서>, axis = 1) 

    주어진 텐서의 1번째 차원의 값들을 다 더해 (값 1개로 만들어서) 그 차원을 없앤다.

sum 외에 prod, min, max, mean, all(논리적 AND), any(논리적 OR), logsumexp 등을 제공

In [21]:
#기본적인 경사하강법으로 최적화
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1)
train_op = optimizer.minimize(cost)

# 텐서플로의 세션을 초기화
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 앞서 구성한 특징과 레이블 데이터를 이용해 학습을 100번 진행
for step in range(100):
    sess.run(train_op, feed_dict = {X:x_data, Y: y_data})
    
    # 학습 도중 10번에 한 번씩 손실값을 출력
    if (step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict = {X:x_data, Y:y_data}))

10 1.1004368
20 1.0769454
30 1.0692252
40 1.064351
50 1.059724
60 1.0550464
70 1.0505656
80 1.0463663
90 1.0431463
100 1.0387527


In [22]:
prediction = tf.argmax(model, axis = 1)
target = tf.argmax(Y, axis = 1)
print('예측값 : ', sess.run(prediction, feed_dict={X:x_data}))
print('실제값 : ',sess.run(target, feed_dict = {Y:y_data}))

예측값 :  [1 1 1 1 1 0]
실제값 :  [0 1 2 0 0 2]


학습된 결과를 확인해 보는 코드를 작성한다.

예측값인 model을 바로 출력하면 확률로 나오기 때문에, **요소 중 가장 큰 값의 인덱스를 찾아주는 argmax 함수**를 사용하여 레이블 값을 출력

one-hot encoding을 거꾸로 한 결과를 만들어 줌

<pre>
[[0 1 0] [1 0 0]] -> [1 0]

[[0.2 0.7 0.1] [0.9 0.1 0.]] -> [1 0]
<code>

In [25]:
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도 : %.2f' %sess.run(accuracy * 100, feed_dict = {X:x_data, Y:y_data}))

정확도 : 16.67


1. 전체 학습 데이터에 대한 예측값과 실측값을 tf.equal 함수로 비교

2. true/false 로 나온 결과를 다시 tf.cast 함수를 이용해 0 과 1로 바꾸어 평균을 내면 간단한 정확도를 구할 수 있음

In [27]:
## 전체 코드
import tensorflow as tf
import numpy as np

x_data = np.array([[0,0], [1,0], [1,1], [0,0], [0,0], [0,1]])
y_data = np.array([[1,0,0], [0,1,0], [0,0,1], [1,0,0], [1,0,0], [0,0,1]])

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W = tf.Variable(tf.random_uniform([2,3], -1., 1.))
b=  tf.Variable(tf.zeros([3]))

L = tf.add(tf.matmul(X, W), b)
L = tf.nn.relu(L)

model = tf.nn.softmax(L)
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis = 1))

optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.01)
train_op = optimizer.minimize(cost)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for step in range(100):
    sess.run(train_op, feed_dict = {X:x_data, Y:y_data})
    
    if (step+1) % 10 == 0:
        print(step+1, sess.run(cost, feed_dict = {X:x_data, Y:y_data}))
        
prediction = tf.argmax(model, axis = 1)
target = tf.argmax(Y, axis = 1)
print('예측값 : ', sess.run(prediction, feed_dict = {X: x_data}))
print('실제값 : ', sess.run(target, feed_dict = {Y:y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도 : %.2f' % sess.run(accuracy * 100, feed_dict = {X:x_data, Y:y_data}))

10 0.94114155
20 0.9381153
30 0.9351527
40 0.93225145
50 0.9294098
60 0.9266253
70 0.9238959
80 0.9212195
90 0.9186346
100 0.9160614
예측값 :  [2 0 2 2 2 2]
실제값 :  [0 1 2 0 0 2]
정확도 : 33.33
